# Sentiment Analysis with LSTM

# 0-) IMPORTS

In [14]:
import matplotlib.pyplot as plt
import numpy             as np
import tensorflow        as tf
from tqdm                import tqdm

from tensorflow.keras.datasets               import mnist
from tensorflow.keras.models                 import Sequential
from tensorflow.keras.layers                 import LeakyReLU, Dense, Reshape, Conv2D, Flatten, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers             import Adam
from tensorflow.keras.callbacks              import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

# 1-) DATA ANALYSIS

In [4]:
(X_train,_), (_,_) = mnist.load_data() #We will only use X_train

In [5]:
X_train = X_train/255.0

In [7]:
print(X_train.shape)
print(len(X_train))

(60000, 28, 28)
60000


In [8]:
#Expand dimensions
X_train = np.expand_dims(X_train, axis=-1)

In [9]:
print(X_train.shape)
print(len(X_train))

(60000, 28, 28, 1)
60000


# 2-) MODELLING

## A-) Create discriminator and generator 

### a-) Discriminator

In [16]:
def build_discriminator():
    model = Sequential()
    #Feature Extraction
    model.add(Conv2D(filters    = 64,
                     kernel_size= (3,3),
                     strides    = 2, # We don't want to reduce data size and loss data.
                     padding    = "same", 
                     input_shape = (28,28,1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(filters    = 128,
                     kernel_size= (3,3),
                     strides    = 2, # We don't want to reduce data size and loss data.
                     padding    = "same"))
    model.add(LeakyReLU(alpha=0.2))
    #Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))

    #Compile
    model.compile(loss="binary_crossentropy",
                  optimizer=Adam(learning_rate = 1e-4,
                                 beta_1        = 0.5),
                  metrics   = [tf.keras.metrics.Precision(name="precision"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.AUC(name="auc"),
                               "accuracy"])
    return model

In [19]:
discriminator_model = build_discriminator()
discriminator_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6273      
                                                                 
Total params: 80,769
Trainable params: 80,769
Non-trai

### b-) Generator

In [23]:
def build_generator():
    model = Sequential()
    
    model.add(Dense(7*7*128,#Flatten dimension
              input_dim=100)) #From noisy vector transform to high level space
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7,7,128)))#transform into matris
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(filters    = 64,
                              kernel_size= (3,3),
                              strides    = 2, # We don't want to reduce data size and loss data.
                              padding    = "same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(filters    = 1,
                              kernel_size= (3,3),
                              strides    = 2, # We don't want to reduce data size and loss data.
                              padding    = "same",
                              activation = "tanh"))
    return model

In [24]:
generator_model = build_generator()
generator_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 6272)              633472    
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 6272)              0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 128)        512       
 ormalization)                                                   
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       73792     
 nspose)                                                         
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 64)       

## B-) Create GAN model = discriminator + generator 

# 3-) MODEL TESTING